# Class Model Visualisation

The idea of class model visualisation arises from the need to somehow open the black box that Neural Networks are when trying to interpret them. This technique  was first developed [[1]](https://arxiv.org/pdf/1312.6034.pdf) to use in image classifying ConvNets. Basically, the idea is to let the network numerically *generate* an image which is representative of the class, that is, the optimal input that the network could receive to classify it inside that class. 

Formally, let $S_c(I)$ be the score of the class $c$, computed by the classification layer of the ConvNet for an image $I$. We would like to find an $L_2$-regularized image, such that the score $S_c$ is high:


$$arg \ max\ S_c(I) - \lambda ||I||_2^2$$

$\lambda$ is the regularisation parameter. A locally-optimal $I$ can be found by back-propagation in the same way the layer weights are optimized during training. This time, however, the weights are fixed and the optimization is performed with respect to the input image. 

It's also important to note that the maximization should be done to the unnormalised class scores $S_c$, since the normalized ones, which have undergone softmax transformation $\big(\frac{exp\ S_c}{\sum_c exp \ S_c}\big)$, can be maximized just by minimizing the scores of other classes.  

In [2]:
from __future__ import division
import numpy as np
import tensorflow as tf
import os
import pandas as pd

ImportError: No module named tensorflow